In [3]:
import pdb

In [2]:
import tensorflow as tf

In [1]:
import os

import numpy as np
import pandas as pd

import sklearn

In [58]:
def fn_getTrainingDataset(strPart, fn_preprocess, strJobsDir, nReaders = 5, nReadThreads = 5, nParseTreads = 5, nShuffleBufferSize = 1000, nBatchSize = 32):
    strSampleFiles = fn_getTrainingSampleFiles(strPart, strJobsDir)
    oDataset = tf.data.Dataset.list_files("jobs/*/samples/positive/sections/" + strPart + "/samples.npy")
    oDataset = oDataset.interleave(lambda strSampleFile: tf.data.TextLineDataset(strSampleFile), cycle_length=nReaders, 
                                  num_parallel_calls=nReadThreads)
    oDataset = oDataset.map(fn_preprocess, num_parallel_calls = nParseTreads)
    oDataset = oDataset.shuffle(nShuffleBufferSize)
    return oDataset.batch(nBatchSize)

In [192]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Dense
from tensorflow import math

import numpy as np

g_nBatchSize = 5

def fn_DLoss(tensorTrue, tensorPred):
        tensorOnes, tensorReal = tensorTrue[:g_nBatchSize, :, :], tensorPred[:g_nBatchSize, :, :]
        tensorZeros, tensorFake = tensorTrue[g_nBatchSize:, :, :], tensorPred[g_nBatchSize:, :, :]
        tensorLossReal = math.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = tensorReal, labels = tensorOnes), 1)
        tensorLossFake = math.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = tensorFake, labels = tensorZeros), 1)
        tensorLoss = tensorLossReal + tensorLossFake
        return math.reduce_mean(tensorLoss)  
def fn_GLoss(tensorTrue, tensorPred):
    tensorLoss = math.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = tensorPred, labels = tensorTrue), 1)
    return math.reduce_mean(tensorLoss)

class GANOperator(object):
    def __init__(self, nLatentDim, nHiddenDimG, nHiddenDimD, nSeqLength, nBatchSize, nFeatures, npNArrSample):
        self.nLatentDim = nLatentDim
        self.nHiddenDimG = nHiddenDimG
        self.nHiddenDimD = nHiddenDimD
        self.nSeqLength = nSeqLength
        self.nBatchSize = nBatchSize
        self.nFeatures = nFeatures
        
        self.npNArrSample = npNArrSample
        
        self.oSeqGenerator = None
        self.oSeqDiscriminator = None
        self.fn_makeGenerator()
        self.fn_makeDiscriminator()
        self.fn_makeDiscriminatorModel()
        self.fn_makeAdversariaModel()
        
    def fn_makeGenerator(self):
        if self.oSeqGenerator:
            return self.oSeqGenerator
        self.oSeqGenerator = Sequential()
        self.oSeqGenerator.add(GRU(self.nHiddenDimG, dropout = 0.1, recurrent_dropout = 0.5, return_sequences = True))
        self.oSeqGenerator.add(Dense(self.nFeatures, activation = math.tanh))
    def fn_makeDiscriminator(self):
        if self.oSeqDiscriminator:
            return self.oSeqDiscriminator
        self.oSeqDiscriminator = Sequential()
        self.oSeqDiscriminator.add(GRU(self.nHiddenDimD, dropout = 0.1, recurrent_dropout = 0.5, return_sequences = True))
        self.oSeqDiscriminator.add(Dense(1))
    
    def fn_makeDiscriminatorModel(self):
        self.oSeqDiscriminatorModel = Sequential()
        self.oSeqDiscriminatorModel.add(self.oSeqDiscriminator)
        oOptimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=0.1)
        self.oSeqDiscriminatorModel.compile(loss=fn_DLoss, optimizer=oOptimizer, metrics=["accuracy"])
    def fn_makeAdversariaModel(self):
        self.oSeqAdversarialModel = Sequential()
        self.oSeqAdversarialModel.add(self.oSeqGenerator)
        self.oSeqAdversarialModel.add(self.oSeqDiscriminator)
        oOptimer = tf.optimizers.Adam()
        self.oSeqAdversarialModel.compile(loss = fn_GLoss, optimizer=oOptimer, metrics=["accuracy"])
    
    def fn_train(self, nEpochs = 2000):
        for epoch in range(nEpochs):
            npNArrSample = self.npNArrSample[np.random.randint(0, self.npNArrSample.shape[0], size = self.nBatchSize), :, :]
            npNoise = np.random.uniform(-1.0, 1.0, size = [self.nBatchSize, self.nSeqLength, self.nLatentDim])
            npNArrFake = self.oSeqGenerator.predict(npNoise)
            npNArrX = np.concatenate((npNArrSample, npNArrFake))
            npNArrY = np.ones([2 * self.nBatchSize, self.nSeqLength, 1])
            npNArrY[self.nBatchSize:, :, :] = 0
            fLossD = self.oSeqDiscriminatorModel.train_on_batch(npNArrX, npNArrY)
            
            npNArrY = np.ones([self.nBatchSize, self.nSeqLength, 1])
            npNoise = np.random.uniform(-1.0, 1.0, size = [self.nBatchSize, self.nSeqLength, self.nLatentDim])
            fLossA = self.oSeqAdversarialModel.train_on_batch(npNoise, npNArrY)

            strMsg = "%d: [D loss: %f, acc: %f]" % (epoch, fLossD[0], fLossD[1])
            strMsg = "%s [A loss: %f, acc: %f]" % (strMsg, fLossA[0], fLossA[1])
            print(strMsg)

In [60]:
npNArrSample = fn_getSamples(False, "input", "jobs")

In [193]:
keras.backend.clear_session()
oGANOperator = GANOperator(nLatentDim=13, nHiddenDimG=100, nHiddenDimD=100, nSeqLength=10, nBatchSize=g_BatchSize, 
                           nFeatures=3, npNArrSample=npNArrSample)

In [194]:
oGANOperator.fn_train(nEpochs=10)

0: [D loss: 1.514886, acc: 0.880000] [A loss: 0.881162, acc: 0.000000]
1: [D loss: 0.627459, acc: 1.000000] [A loss: 1.197199, acc: 0.000000]
2: [D loss: 0.455534, acc: 1.000000] [A loss: 1.629318, acc: 0.000000]
3: [D loss: 0.467883, acc: 0.990000] [A loss: 2.484230, acc: 0.000000]
4: [D loss: 0.309302, acc: 1.000000] [A loss: 2.279864, acc: 0.000000]
5: [D loss: 0.256230, acc: 1.000000] [A loss: 3.116179, acc: 0.000000]
6: [D loss: 0.245415, acc: 1.000000] [A loss: 3.506050, acc: 0.000000]
7: [D loss: 0.186093, acc: 1.000000] [A loss: 3.124023, acc: 0.000000]
8: [D loss: 0.211709, acc: 1.000000] [A loss: 4.235771, acc: 0.000000]
9: [D loss: 0.217565, acc: 1.000000] [A loss: 4.427356, acc: 0.000000]
